# AWS Bedrock Inline Agents with Llama Models

This notebook demonstrates building intelligent AI agents using Amazon Bedrock's inline agent capabilities powered by Meta's Llama foundation models. The agent architecture enables sophisticated automation workflows without requiring pre-deployed infrastructure, making it ideal for rapid agent prototyping and development.

![Email Analysis Agent](email_agent_workflow.png)

# Environment Setup for Agent Development
## Agent Development Environment Setup

Amazon Bedrock provides managed access to Meta's Llama models, eliminating the need for model hosting infrastructure while enabling enterprise-grade agent development. The boto3 SDK provides comprehensive integration with Bedrock's agent runtime services.


In [ ]:
# Installing boto3 provides the AWS SDK for Python
# This enables programmatic access to Amazon Bedrock services
# including the bedrock-agent-runtime client for inline agents
%pip install boto3

# Agent Runtime Configuration
## AWS Bedrock Agent Runtime Integration

The configuration establishes connections to Amazon Bedrock's agent runtime services, which provide the execution environment for inline agents. Regional configuration ensures optimal performance and compliance with data residency requirements for agent operations.

In [1]:
import boto3
from botocore.config import Config  # Regional configuration for optimal performance
import json  # JSON handling for agent request/response serialization
from IPython.display import display, HTML, Markdown  # Rich output formatting for agent responses
from typing import Dict, Any
import json

# Helper function to initialize Bedrock clients with regional configuration
def get_bedrock_clients(region="us-west-2"):
    config = Config(region_name=region)  # Regional config for latency optimization
    bedrock_runtime = boto3.client('bedrock-runtime', config=config)  # Direct model invocation
    bedrock_agent_runtime = boto3.client('bedrock-agent-runtime', config=config)  # Agent execution
    return bedrock_runtime, bedrock_agent_runtime

# Response formatter for agent outputs with structured display
def display_response(response, response_type=""):
    """Pretty print agent responses with markdown formatting"""
    if response_type:
        display(Markdown(f"## {response_type}"))
    
    completion = response.get('completion', '')  # Extract agent completion text
    display(Markdown(completion))
    
    # Display additional metadata from agent execution
    additional_data = response.get('additionalData', {})
    if additional_data:
        display(Markdown("### Additional Information"))
        display(Markdown(f"```json\n{json.dumps(additional_data, indent=2)}\n```"))


# Llama-Powered Inline Agent Architecture
## Agent Configuration and Runtime Setup

This cell demonstrates the initialization of Amazon Bedrock's agent runtime client and the configuration structure for inline agents. Amazon Bedrock inline agents allow you to configure and invoke agents at runtime without pre-deployment, providing flexibility to specify agent capabilities like foundation models, instructions, and action groups at invocation time [[1]](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-action-create.html). The configuration defines a comprehensive agent structure with multiple specialized operations for email processing, utilizing Meta's Llama 3.3 70B model for advanced natural language understanding and reasoning capabilities. Each action within the agent includes detailed parameter schemas that ensure reliable execution and proper input validation, while the modular action group design enables scalable agent architectures that can be extended with additional capabilities as business requirements evolve.

**Llama Model Advantages for Agents:**
- Superior reasoning capabilities for complex decision-making
- Strong instruction following for reliable agent behavior  
- Multilingual support for global agent deployments
- Cost-effective inference through Bedrock's managed infrastructure

In [2]:
# Configure AWS client for agent runtime operations
bedrock_agent_runtime = boto3.client(
    service_name='bedrock-agent-runtime',  # Specialized client for agent execution
    region_name='us-west-2'  # Region selection affects latency and model availability
)

import uuid  # For generating unique session identifiers
from datetime import datetime  # Timestamp handling for agent sessions

def create_email_agent_config():
    return {
        "name": "EmailAnalysisAgent",# Agent identifier for tracking and logging
        "description": "An agent for analyzing customer emails with multiple capabilities and department routing",

        # Comprehensive instructions that guide Llama model behavior
        "instructions": """You are an expert email assistant specializing in customer service. Your capabilities include:
        1. Identifying appropriate departments for customer inquiries
        2. Summarizing emails into clear, concise points
        3. Drafting professional responses with appropriate tone
        4. Categorizing emails by priority and type
        5. Extracting key action items and deadlines
        
        Department Categories:
        - Customer Support (general inquiries, account access)
        - Technical Support (technical issues, bugs, system errors)
        - Sales (pricing, purchases, upgrades)
        - Billing (payments, invoices, refunds)
        - Product Information (features, documentation)
        
        Always maintain a professional tone and ensure responses are well-structured.""",

        # Llama 3.3 70B model specification - optimized for complex reasoning
        "foundationModel": "us.meta.llama3-3-70b-instruct-v1:0",
        # Action groups define the agent's capabilities and available operations
        "actionGroups": [
            {
                "actionGroupName": "EmailOperations",# Logical grouping of related actions
                "actionGroupState": "DISABLED",
                "description": "Email analysis operations",
                "actions": [
                    {
                        "name": "identifyDepartment",# Specific agent capability
                        "description": "Identify the appropriate department for handling the customer email",
                        # JSON schema defines expected input structure for reliable agent execution
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "emailContent": {
                                    "type": "string",
                                    "description": "The content of the email to analyze"
                                }
                            },
                            "required": ["emailContent"]# Ensures required inputs are provided
                        }
                    },
                    # Additional actions follow the same pattern...
                    {
                        "name": "summarizeEmail",
                        "description": "Create a concise summary of the customer email content",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "emailContent": {
                                    "type": "string",
                                    "description": "The content of the email to summarize"
                                }
                            },
                            "required": ["emailContent"]
                        }
                    },
                    {
                        "name": "categorizeEmail",
                        "description": "Categorize customer email by department, priority and type",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "emailContent": {
                                    "type": "string",
                                    "description": "The content of the email to categorize"
                                }
                            },
                            "required": ["emailContent"]
                        }
                    },
                    {
                        "name": "extractActionItems",
                        "description": "Extract action items and deadlines from the customer email",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "emailContent": {
                                    "type": "string",
                                    "description": "The content of the email to extract action items from"
                                }
                            },
                            "required": ["emailContent"]
                        }
                    },
                    {
                        "name": "draftResponse",
                        "description": "Draft a professional response to the customer email",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "emailContent": {
                                    "type": "string",
                                    "description": "The original email content"
                                },
                                "department": {
                                    "type": "string",
                                    "description": "The department handling the response"
                                },
                                "tone": {
                                    "type": "string",
                                    "enum": ["formal", "casual", "neutral"],
                                    "description": "The tone of the response"
                                }
                            },
                            "required": ["emailContent", "department", "tone"]
                        }
                    },
                    {
                        "name": "analyzeUrgency",
                        "description": "Analyze the urgency and priority of the customer email",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "emailContent": {
                                    "type": "string",
                                    "description": "The content of the email to analyze"
                                }
                            },
                            "required": ["emailContent"]
                        }
                    },
                    {
                        "name": "suggestNextSteps",
                        "description": "Suggest next steps for handling the customer inquiry",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "emailContent": {
                                    "type": "string",
                                    "description": "The content of the email"
                                },
                                "department": {
                                    "type": "string",
                                    "description": "The identified department"
                                },
                                "priority": {
                                    "type": "string",
                                    "description": "The assessed priority level"
                                }
                            },
                            "required": ["emailContent", "department", "priority"]
                        }
                    }
                ]
            }
        ]
    }

### Amazon Bedrock Inline Agent Implementation

This function demonstrates the core implementation of Amazon Bedrock inline agents, which provide serverless AI automation capabilities that can be configured and invoked dynamically at runtime without requiring pre-deployed infrastructure [[1]](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-create-inline.html). The implementation serves as the central processing engine for Llama model interactions, utilizing structured prompt engineering where different analytical tasks such as department identification, summarization, and categorization are handled through operation-specific prompts that guide the foundation model's behavior to produce consistent, structured responses. Session management is implemented through unique UUID generation to enable conversation tracking and context maintenance across multiple agent interactions, while the function processes streaming responses from the Bedrock agent runtime by decoding binary chunks and combining them into complete responses with comprehensive error handling for robust production operation [[2]](https://docs.aws.amazon.com/bedrock/latest/userguide/service_code_examples_bedrock-agent-runtime.html). This approach leverages the interactive notebook environment's capability to maintain session state and variable tracking, allowing for iterative development and testing of agent behaviors within the managed AWS infrastructure [[3]](https://docs.aws.amazon.com/sagemaker/latest/dg/nbi.html).



In [3]:
# Core agent processing function that handles Llama model interaction
def process_email(email_content: str, operation_type: str, agent_runtime, **kwargs) -> Dict[str, Any]:
    try:
        # Generate unique session ID for tracking agent conversations
        session_id = f"email-session-{uuid.uuid4()}"
        
        # Operation-specific prompts that guide Llama model behavior
        operation_prompts = {
            "identify_department": """
            Please analyze this customer email and:
            1. Identify the most appropriate department (Customer Support, Technical Support, Sales, Billing, Product Information)
            2. Provide reasoning for the department selection
            3. List any additional departments that might need to be involved
            4. Specify the primary customer concern
            """,

            # Additional prompts customize agent behavior for different operations...
            "summarize": """
            Please provide a comprehensive summary of this customer email including:
            - Customer's main concern or request
            - Relevant department(s)
            - Key points and details
            - Any deadlines or important dates
            - Required actions
            Format the summary in bullet points.
            """,
            
            "categorize": """
            Please categorize this customer email considering:
            1. Department (e.g., Technical Support, Customer Service, Sales, Billing)
            2. Type (e.g., Complaint, Inquiry, Request, Feedback)
            3. Priority Level (Critical, High, Medium, Low)
            4. Action Required (Immediate Action, Action Required, Information Only)
            5. Time Sensitivity (Urgent, Time-bound, Regular)
            Provide reasoning for each categorization.
            """,
            
            "extractActionItems": """
            Please extract and list all action items from this email:
            1. Identify specific tasks or actions required
            2. Specify who needs to take action (if mentioned)
            3. Note any deadlines or time constraints
            4. Prioritize the actions (High/Medium/Low)
            5. Indicate if any follow-up is needed
            Format as a structured list with clear, actionable items.
            """,
            
            "draftResponse": """
            Write a complete email response that includes:
            1. Proper greeting with customer's name (if available)
            2. Direct acknowledgment of their concerns
            3. Clear solutions or next steps for each point raised
            4. Professional and empathetic tone
            5. Appropriate signature and department name
            Write ONLY the email response - no explanations or analysis.
            """,
            
            "analyzeUrgency": """
            Please analyze the urgency of this email by:
            1. Assessing the immediacy of the request
            2. Identifying any explicit or implicit deadlines
            3. Evaluating the potential impact of delayed response
            4. Recommending a response timeframe
            5. Highlighting any critical factors affecting urgency
            Provide a clear urgency rating (Critical/High/Medium/Low) with justification.
            """,
            
            "suggestNextSteps": """
            Please suggest next steps for handling this customer inquiry:
            1. Immediate actions required
            2. Follow-up tasks needed
            3. Escalation requirements (if any)
            4. Timeline recommendations
            5. Success metrics or completion criteria
            Format as a prioritized action plan.
            """
        }

        # Build prompt based on operation type
        if operation_type == "draftResponse":
            # Handle draftResponse with optional parameters
            department = kwargs.get('department', 'Customer Support')
            tone = kwargs.get('tone', 'professional')
            
            prompt = f"""
            Write a professional email response for the {department} department using a {tone} tone.
            
            Original Customer Email:
            {email_content}
            
            {operation_prompts.get(operation_type)}
            """
            
            instruction = "You are a professional customer service representative. Write a complete, ready-to-send email response."
            
        elif operation_type == "suggestNextSteps":
            # Handle suggestNextSteps with optional parameters
            department = kwargs.get('department', 'Customer Support')
            priority = kwargs.get('priority', 'Medium')
            
            prompt = f"""
            Operation: {operation_type}
            Department: {department}
            Priority Level: {priority}
            
            {operation_prompts.get(operation_type)}
            
            Email Content:
            {email_content}
            """
            
            instruction = """You are an expert customer email analyzer. 
                            Process the email according to the specified operation type. 
                            Focus on identifying customer needs, appropriate departments, and required actions. 
                            Provide detailed, structured responses with clear formatting."""
        else:
            # Handle all other operations normally
            prompt = f"""
            Operation: {operation_type}
            
            {operation_prompts.get(operation_type, "Please analyze this customer email.")}
            
            Email Content:
            {email_content}
            """
            
            instruction = """You are an expert customer email analyzer. 
                            Process the email according to the specified operation type. 
                            Focus on identifying customer needs, appropriate departments, and required actions. 
                            Provide detailed, structured responses with clear formatting."""
         # Request structure for inline agent invocation
        request = {
            "foundationModel": "us.meta.llama3-3-70b-instruct-v1:0",# Llama model specification
            "instruction": instruction,# High-level agent guidance
            "inputText": prompt,# Specific task prompt
            "sessionId": session_id,# Session tracking for conversation continuity
            "enableTrace": True# Enable detailed execution tracing for debugging
        }

        # Invoke the inline agent with streaming response handling
        response = agent_runtime.invoke_inline_agent(**request)

        # Process streaming response chunks from Llama model
        result = []
        for event in response.get('completion', []):
            if 'chunk' in event:
                chunk = event['chunk']['bytes'].decode()# Decode binary response chunks
                result.append(chunk)
        
        return {
            'status': 'success',
            'operation': operation_type,
            'content': ''.join(result)# Combine chunks into complete response
        }

    except Exception as e:
        # Comprehensive error handling for agent execution failures
        return {
            'status': 'error',
            'operation': operation_type,
            'error': str(e)
        }

# Agent Test Scenarios
## Comprehensive Agent Testing Framework

The test scenarios demonstrate how Llama-powered agents can handle diverse business contexts with sophisticated reasoning capabilities. Each scenario tests different aspects of the agent's analytical and decision-making abilities, showcasing Llama's strength in complex reasoning tasks.

**Agent Reasoning Capabilities:**
- Multi-step analysis with logical progression
- Context-aware decision making based on business rules
- Adaptive responses based on content complexity and urgency
- Structured output generation following specific formats

In [4]:
# Test cases representing realistic business scenarios for agent processing
test_emails = {
    # Product enhancement scenario testing agent's categorization
    "feature_request": """
    Subject: New Feature Suggestion for Dashboard
    
    Hi Product Team,
    
    Our team has been using your analytics dashboard for 6 months now, and we have a feature request.
    It would be extremely helpful if we could:
    1. Schedule reports to be sent automatically every Monday at 9 AM
    2. Customize the dashboard layout for different team members
    3. Export data in CSV format
    
    We need this functionality by Q2 as we're expanding our team.
    
    Best regards,
    Mark Thompson
    Project Manager
    """,
    # Critical issue scenario testing urgency analysis
    "technical_support": """
    Subject: Urgent - API Integration Issues
    
    Hello Support,
    
    We're experiencing critical issues with the API integration:
    - Authentication tokens are expiring too quickly
    - Receiving 503 errors during peak hours (2-4 PM EST)
    - Webhook notifications are delayed by 30+ minutes
    
    Impact:
    - 500+ transactions affected
    - Customer complaints increasing
    - Revenue impact estimated at $10k/day
    
    Please provide immediate assistance. Our technical team is available for a call anytime.
    
    Reference: INC-789456
    
    Regards,
    Janet Chen
    Technical Lead
    """,
    # Financial inquiry testing department routing logic
    "billing_inquiry": """
    Subject: Invoice Clarification and Payment Update
    
    Dear Billing Department,
    
    Regarding Invoice #INV-2024-0123:
    1. There seems to be a discrepancy in the user count (we have 45, but charged for 50)
    2. The premium support fee wasn't included in our contract
    3. We need to update our payment method before the next billing cycle (March 1st)
    
    Please review and send an updated invoice by Friday. Also, could you schedule a call 
    to discuss our annual renewal options?
    
    Thanks,
    Sarah Miller
    Finance Director
    """
}

Each test email includes:
- Realistic business context and terminology
- Multiple decision points for agent analysis
- Varying urgency levels to test prioritization logic
- Specific details that require agent reasoning capabilities

# Agent Operation Testing
## Systematic Agent Validation

The testing framework validates each agent operation independently and in combination, demonstrating how Llama models maintain consistency across different analytical tasks while adapting their reasoning approach based on the specific operation requirements.



In [5]:
def analyze_operation_results(all_results: Dict):
    """
    Analyze and display results from all operations
    
    Args:
        all_results (Dict): Results from all test operations
    """
    print("\nOPERATIONS ANALYSIS SUMMARY")
    print("=" * 80)
    
    for email_type, operations in all_results.items():
        print(f"\nEmail Type: {email_type.upper().replace('_', ' ')}")
        print("-" * 50)
        
        for operation, result in operations.items():
            success = result['status'] == 'success'
            print(f"\nOperation: {operation}")
            print(f"Status: {'✓ Success' if success else '✗ Failed'}")
            if not success:
                print(f"Error: {result.get('error', 'Unknown error')}")
            print("-" * 30)
def run_specific_test(email_type: str = "feature_request", 
                     operation: str = "summarize") -> Dict:
    """
    Run a test for a specific email type and operation
    
    Args:
        email_type (str): Type of test email to use
        operation (str): Operation to perform
        
    Returns:
        Dict: Result of the operation
    """
    if email_type not in test_emails:
        print(f"Email type '{email_type}' not found")
        return
    
    print(f"\nTesting {operation.upper()} for {email_type.upper()}")
    print("-" * 50)
    print("Email Content:")
    print(test_emails[email_type].strip())
    print("-" * 50)
    
    result = process_email(test_emails[email_type], operation, bedrock_agent_runtime)
    print("\nResult:")
    print(json.dumps(result, indent=2))
    return result



This cell demonstrates targeted testing of Amazon Bedrock inline agents by executing a specific operation on a predefined email scenario, showcasing how agents can be configured and invoked at runtime without pre-deployment [[1]](https://docs.aws.amazon.com/bedrock/latest/userguide/inline-agent-invoke.html). The `run_specific_test` function isolates individual agent capabilities for focused analysis, allowing developers to validate specific operations like action item extraction from technical support emails while leveraging the Llama model's reasoning capabilities through structured prompt engineering and session management.

In [6]:
# Test a specific operation on a specific email type
result = run_specific_test(
    email_type="technical_support",
    operation="extractActionItems"
)


Testing EXTRACTACTIONITEMS for TECHNICAL_SUPPORT
--------------------------------------------------
Email Content:
Subject: Urgent - API Integration Issues
    
    Hello Support,
    
    We're experiencing critical issues with the API integration:
    - Authentication tokens are expiring too quickly
    - Receiving 503 errors during peak hours (2-4 PM EST)
    - Webhook notifications are delayed by 30+ minutes
    
    Impact:
    - 500+ transactions affected
    - Customer complaints increasing
    - Revenue impact estimated at $10k/day
    
    Please provide immediate assistance. Our technical team is available for a call anytime.
    
    Reference: INC-789456
    
    Regards,
    Janet Chen
    Technical Lead
--------------------------------------------------

Result:
{
  "status": "success",
  "operation": "extractActionItems",
  "content": "To address the user's request, I need to call the \"extractActionItems\" tool. However, before making the call, I need to confirm the em

### Comprehensive Agent Operation Testing

This cell implements a systematic testing framework that validates multiple agent operations sequentially on the same email content. Each operation in the test suite evaluates distinct capabilities of the Llama model through Amazon Bedrock inline agents, including content comprehension, classification reasoning, structured information extraction, content generation with tone adaptation, and priority assessment with business logic.


In [7]:
def test_all_operations(email_content: str) -> Dict:
    """
    Comprehensive agent testing across all defined operations
    Tests Llama model's ability to maintain context while switching between different analytical modes
    """
    operations = [
        "summarize",        # Tests content comprehension and synthesis
        "categorize",       # Tests classification and reasoning abilities  
        "extractActionItems", # Tests structured information extraction
        "draftResponse",    # Tests content generation and tone adaptation
        "analyzeUrgency"    # Tests priority assessment and business logic
    ]
    
    results = {}
    for operation in operations:
        # Each operation tests different aspects of Llama model capabilities
        result = process_email(email_content, operation, bedrock_agent_runtime)
        results[operation] = result
        # Results demonstrate agent consistency and reasoning quality
    return results

This code cell demonstrates a comprehensive testing approach for Amazon Bedrock inline agents, systematically validating agent capabilities across multiple email scenarios and operations.

In [8]:
# Run all operations on all test emails
all_results = {}
for email_type, content in test_emails.items():
    print(f"\nTesting Email Type: {email_type.upper().replace('_', ' ')}")
    print("=" * 80)
    all_results[email_type] = test_all_operations(content)

# Analyze results
analyze_operation_results(all_results)


Testing Email Type: FEATURE REQUEST

Testing Email Type: TECHNICAL SUPPORT

Testing Email Type: BILLING INQUIRY

OPERATIONS ANALYSIS SUMMARY

Email Type: FEATURE REQUEST
--------------------------------------------------

Operation: summarize
Status: ✓ Success
------------------------------

Operation: categorize
Status: ✓ Success
------------------------------

Operation: extractActionItems
Status: ✓ Success
------------------------------

Operation: draftResponse
Status: ✓ Success
------------------------------

Operation: analyzeUrgency
Status: ✓ Success
------------------------------

Email Type: TECHNICAL SUPPORT
--------------------------------------------------

Operation: summarize
Status: ✓ Success
------------------------------

Operation: categorize
Status: ✓ Success
------------------------------

Operation: extractActionItems
Status: ✓ Success
------------------------------

Operation: draftResponse
Status: ✓ Success
------------------------------

Operation: analyzeUrgen

The all_results dictionary serves as a comprehensive collection storing test outcomes for each email type and operation combination. The nested loop structure processes each test email through all defined agent operations, creating a matrix of results that demonstrates agent consistency across different scenarios.

The test_all_operations(content) function call executes the complete suite of agent capabilities on each email type, testing the Llama model's ability to maintain analytical quality while switching between different operational modes. This approach mirrors testing methodologies used in AWS services where comprehensive validation ensures reliable performance across diverse use cases.

The formatted print statements provide clear visual separation between test scenarios, making it easier to track testing progress and identify specific test cases during execution. The analyze_operation_results(all_results) function call processes the complete test matrix to generate summary insights about agent performance patterns and potential issues.

This testing framework enables systematic validation of agent capabilities, ensuring that the Llama-powered inline agent maintains consistent performance across different email types and analytical operations. The comprehensive approach helps identify edge cases and validates the robustness of the agent configuration for production deployment scenarios.

# Agent Response Visualization
## Professional Agent Output Presentation

The visualization system transforms Llama model responses into structured, professional output suitable for business applications. This demonstrates how agent responses can be formatted for different stakeholders and use cases while preserving the analytical depth provided by Llama's reasoning capabilities.


### Result Formatting and Display Functions
The `format_operation_result` function transforms raw agent responses into visually distinct, color-coded HTML presentations tailored to each operation type. This function handles error states with red-highlighted error messages and applies operation-specific styling including blue backgrounds for urgency analysis, green for categorization, pink for summaries, yellow for action items, and gray for draft responses.

The `display_email_analysis` function creates comprehensive analysis presentations by combining the original email content display with sequential formatting of all operation results. This function establishes the visual hierarchy with email type headers, original content preservation in preformatted blocks, and systematic presentation of each agent operation's output through the formatting function.

In [9]:
def format_operation_result(result: Dict[str, Any], operation: str):
    """
    Format operation results for better display in Jupyter notebook
    """
    if result['status'] != 'success':
        display(HTML(
            f"<div style='background-color: #ffe6e6; padding: 10px; border-left: 5px solid #ff4444; margin: 10px 0;'>"
            f"<h3 style='color: #cc0000;'>❌ Error in {operation}</h3>"
            f"<p>{result.get('error', 'Unknown error')}</p>"
            "</div>"
        ))
        return

    # Different formatting based on operation type
    if operation == "analyzeUrgency":
        content = result['content']
        display(HTML(
            f"<div style='background-color: #f0f7ff; padding: 15px; border-radius: 5px; margin: 10px 0;'>"
            f"<h3 style='color: #2c5282;'>📊 Urgency Analysis</h3>"
            f"<div style='white-space: pre-wrap;'>{content.replace(chr(10), '<br>')}</div>"
            "</div>"
        ))

    elif operation == "categorize":
        content = result['content']
        display(HTML(
            f"<div style='background-color: #f0fff4; padding: 15px; border-radius: 5px; margin: 10px 0;'>"
            f"<h3 style='color: #276749;'>🏷️ Email Categorization</h3>"
            f"<div style='white-space: pre-wrap;'>{content.replace(chr(10), '<br>')}</div>"
            "</div>"
        ))

    elif operation == "summarize":
        content = result['content']
        display(HTML(
            f"<div style='background-color: #fff5f7; padding: 15px; border-radius: 5px; margin: 10px 0;'>"
            f"<h3 style='color: #702459;'>📝 Email Summary</h3>"
            f"<div style='white-space: pre-wrap;'>{content.replace(chr(10), '<br>')}</div>"
            "</div>"
        ))

    elif operation == "extractActionItems":
        content = result['content']
        display(HTML(
            f"<div style='background-color: #fffaf0; padding: 15px; border-radius: 5px; margin: 10px 0;'>"
            f"<h3 style='color: #744210;'>✅ Action Items</h3>"
            f"<div style='white-space: pre-wrap;'>{content.replace(chr(10), '<br>')}</div>"
            "</div>"
        ))

    elif operation == "draftResponse":
        content = result['content']
        display(HTML(
            f"<div style='background-color: #f7fafc; padding: 15px; border-radius: 5px; margin: 10px 0;'>"
            f"<h3 style='color: #2a4365;'>✉️ Draft Response</h3>"
            f"<div style='white-space: pre-wrap; font-family: Arial, sans-serif;'>{content.replace(chr(10), '<br>')}</div>"
            "</div>"
        ))

def display_email_analysis(email_type: str, email_content: str, results: Dict):
    """
    Display comprehensive analysis for an email
    """
    display(HTML(
        f"<div style='background-color: #edf2f7; padding: 20px; border-radius: 10px; margin: 20px 0;'>"
        f"<h2 style='color: #1a365d;'>📧 Analysis for: {email_type.upper().replace('_', ' ')}</h2>"
        f"<div style='background-color: white; padding: 15px; border-radius: 5px; margin: 10px 0;'>"
        f"<h3 style='color: #2d3748;'>Original Email:</h3>"
        f"<pre style='background-color: #f7fafc; padding: 10px; border-radius: 5px;'>{email_content.strip()}</pre>"
        "</div>"
        "</div>"
    ))

    for operation, result in results.items():
        format_operation_result(result, operation)

### Advanced Agent Testing with Rich Display

This function demonstrates the enhanced testing approach that combines comprehensive agent operations with professional result presentation. The improved testing framework provides immediate visual feedback and structured analysis presentation suitable for demonstration and educational purposes.

In [10]:
def run_improved_test(email_type: str = "feature_request"):
    """
    Run all operations on a specific email type with improved display
    """
    if email_type not in test_emails:
        display(HTML(
            f"<div style='background-color: #ffe6e6; padding: 10px; border-left: 5px solid #ff4444;'>"
            f"<p>Email type '{email_type}' not found</p>"
            "</div>"
        ))
        return

    email_content = test_emails[email_type]
    results = {}
    
    operations = [
        "summarize",
        "categorize",
        "extractActionItems",
        "draftResponse",
        "analyzeUrgency"
    ]
    
    for operation in operations:
        results[operation] = process_email(email_content, operation, bedrock_agent_runtime)
    
    display_email_analysis(email_type, email_content, results)
    return results

## Comprehensive Agent Testing Execution

This cell executes comprehensive testing of all agent operations on a single email scenario, demonstrating systematic validation of Amazon Bedrock inline agent capabilities across multiple analytical tasks. The `run_improved_test` function processes the "technical_support" email scenario through all defined operations including summarization, categorization, action item extraction, response drafting, and urgency analysis.



In [12]:
# Test a single email type
results = run_improved_test("technical_support")

## Next Steps: Expanding Your Bedrock Inline Agent with Llama Models

Now that you've built a functional email processing system with Amazon Bedrock inline agents powered by Llama models, you can explore integrating additional AWS services to enhance your agent's capabilities and create more sophisticated workflows for educational experimentation. Amazon Bedrock manages the underlying complexity including prompt engineering, memory, monitoring, encryption, user permissions, and API invocation, allowing you to focus on building functionality rather than infrastructure [[1]](https://docs.aws.amazon.com/bedrock/latest/userguide/agents.html).

### Exploring Advanced Agent Configurations with Llama Models

Your inline agent implementation demonstrates the power of combining Llama's reasoning capabilities with Amazon Bedrock's managed infrastructure. The platform exposes default prompt templates for pre-processing, orchestration, knowledge base response generation, and post-processing, which you can optionally customize to modify your agent's behavior at each step of its sequence [[2]](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-how.html). This flexibility is particularly valuable when working with Llama models, as you can fine-tune the prompting strategies to leverage Llama's specific strengths in reasoning and instruction following.

For expanding your educational exploration, consider experimenting with action groups that define actions your agent can help end users perform, where each action group includes the parameters that the agent must elicit from the end-user [[3]](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-prereq.html). You can also explore Lambda function integration, where your agent can use Lambda functions by attaching resource-based policies to provide the necessary permissions [[4]](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-lambda.html). Additionally, Amazon Bedrock provides comprehensive code examples that demonstrate how to perform individual Amazon Bedrock Agents actions with AWS SDKs, offering practical implementation patterns you can adapt for your own experiments [[5]](https://docs.aws.amazon.com/bedrock/latest/userguide/service_code_examples_bedrock-agent_actions.html).

The inline agent approach you've implemented provides an excellent foundation for understanding how Llama models can orchestrate interactions between foundation models, data sources, software applications, and user conversations, automatically calling APIs to take actions and invoking knowledge bases to supplement information [[1]](https://docs.aws.amazon.com/bedrock/latest/userguide/agents.html).






